# Using `pymf6` Interactively

You can run a MODFLOW6 model interactively.
For example, in a Jupyter Notebook.

## Setup

First, change into the directory of your MODFLOW6 Model:

`[1] %cd c:\Users\me\path\to\examples\ex02-tidal`

User your full path instead.

In [ ]:
%%capture

#  ## Hidden `nbsphinx' cell

# Set LD_LIBRARY_PATH for readthedocs
# Makes it find libgfortran on Linux

import os
from os.path import split, join

cur_path = %pwd
pymf6_path = join(split(split(cur_path)[0])[0], 'pymf6')
old_lib_path = os.getenv('LD_LIBRARY_PATH', '')
if old_lib_path:
    new_lib_path = ':'.join([old_lib_path, pymf6_path])
else:
    new_lib_path = pymf6_path
%set_env LD_LIBRARY_PATH=$new_lib_path

# Change to model directory

%cd -q ../examples/ex02-tidal/

The directory `ex02-tidal` contains all files needed to run a MODFLOW6 model.
The model `ex02-tidal` is one of the example models that come with MODFLOW6.

Now, import the class `MF6` from the module `pymf6.threaded`:

In [ ]:
from pymf6.threaded import MF6

make an instance of this class:

In [ ]:
mf6 = MF6()

## Meta Data

This instance offers meta data such as the names of the models:

In [ ]:
mf6.simulation.model_names

or the time unit:

In [ ]:
mf6.simulation.time_unit

as well as the associated time multiplier to convert to seconds:

In [ ]:
mf6.simulation.time_multiplier

## Temporal Discretization (TDIS)

Data about stress periods and time steps are also available.
Get the number of stress periods:

In [ ]:
mf6.simulation.TDIS.NPER

or the total simulation time (in days in this case):

In [ ]:
mf6.simulation.TDIS.TOTALSIMTIME

In [ ]:
mf6.simulation.TDIS.var_names

## Simulations

A MODFLOW6 model can contain multiple simulations.
To date all examples that come with MODFLOW6 only have one simulation.
The simulations are assembled in a list.
Get the first element of that list:

In [ ]:
sim1 = mf6.simulation.solution_groups[0]

In [ ]:
sim1

As you can see this solution has one package and 68 packages.
You can list all package name.
One in our case:

In [ ]:
sim1.package_names

All internal MODFLOW6 names are all upper case such as `IMSLINEAR`.
New names introduced by `pymf6` are all lower case, with underscores for longer names such as `package_names`.

Now you can access this package via Python's attribute access:

In [ ]:
sim1.IMSLINEAR

This is equivalent to the dictionary-like key access (called `__getitem__`):

In [ ]:
sim1['IMSLINEAR']

This first approach has the advantage that you can take advantage of tab completion, i.e. after the dot press the `<TAB>` key in the Notebook (this might be a different key in an other tool), you will get a list of all possible names.
After you start typing, the list narrows down to the names that start with the letters you typed.

The second approach can be useful, if you now the name of the package or variable.
For example, `var_names` contains all variable names in a list:

In [ ]:
len(sim1.var_names)

Let's display the first five names:

In [ ]:
sim1.var_names[:5]

Now show the names **and** the values of this first five names: 

In [ ]:
for name in sim1.var_names[:5]:
    print(name, sim1[name].value)

A variable itself displays a bit more verbose in the Notebook:

In [ ]:
sim1.ID

The printed strings also works without the notebook:

In [ ]:
print(sim1.ID)

A package also has variables.
Let's create a shorter name:

In [ ]:
ims = sim1.IMSLINEAR

In [ ]:
ims

and access the variables names:

In [ ]:
len(ims.var_names)

In [ ]:
ims.var_names[:5]

You can also keep the long name (remember to use the `<TAB>` key to get name suggestions):

In [ ]:
sim1.IMSLINEAR.IPC.value

## Models

Typically, a solution is not so interesting.
More frequently, you might want to get more information about a model or even change model variable values at run time.
Again, there can be several models.
So far, MODFLOW6 supports only one model.
This might change in the future.
Therefore, `pymf6` already works with a list of models (with of length one so far ;).
Take the first model from the models list of the first simulation:

In [ ]:
model = mf6.simulation.models[0]

In [ ]:
model

It has 17 packages and 43 variables.
Again, all names are contained in lists:

In [ ]:
len(model.package_names)

In [ ]:
len(model.var_names)

## Packages

Let's access something more interesting.
For example the Structured Discretization (DIS) package:

In [ ]:
model.DIS

Get the number of layers.
The full variable:

## Variables

In [ ]:
model.DIS.NLAY

Get only the number:

In [ ]:
model.DIS.NLAY.value

This is the shape of the grid:

In [ ]:
model.DIS.MSHAPE

Also available as a NumPy array:

In [ ]:
model.DIS.MSHAPE.value

The bottom variable has more elements (number of elements = layer * row * column):

In [ ]:
model.DIS.BOT

The `value` holds a NumPy array:

In [ ]:
model.DIS.BOT.value

The bottom comes as a one-dimensional array.
But it represents a three-dimensional structure.
For all variables for this is applicable, `value_3d` gives a 3D-view:

In [ ]:
model.DIS.BOT.value_3d

You can also access elements for 3D-variables via the one-based `layer`, `row`, `column` index.
This gives the element in layer 1, row 1, and column 1:

In [ ]:
model.DIS.BOT.get_value_by_lrc(1, 1, 1)

## Setting Values

You cannot only read the values of variables.
It is possible to set all values.

Create a shorter name for the bottom data:

In [ ]:
bot = model.DIS.BOT

In [ ]:
bot

Change the first value:

In [ ]:
bot[0] = 7

In [ ]:
bot

Now, a short name for the 3D-view:

In [ ]:
bot_3d = bot.value_3d
bot_3d

Changing the value of an element:

In [ ]:
bot_3d[0, 0, 0] = 6.5
bot_3d

has the same effect:

In [ ]:
bot

## Doing a Time Step

This make MODFLOW6 to calculate the next time step:

In [ ]:
mf6.next_step()

Instead of calling this method again and again till the end, call `run_to_end()`:

In [ ]:
# mf6.run_to_end()

This will kill the current kernel because Fortran calls `STOP`, which terminates the current process.